In [1]:
import os
import csv
from countries_languages import country_to_language

import pandas as pd
from difflib import get_close_matches
import numpy as np
import re
from transliterate import translit
from unidecode import unidecode
import Levenshtein
import requests
import bs4
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from datetime import datetime

from urllib.parse import urlsplit

import wikipediaapi

import wikipedia

from urllib.parse import urlparse

import difflib

from datetime import datetime

import calendar

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import AssemblyHelpers
#from AssemblyHelpers import find_money_info_from_name



/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/AssemblyHelpers.py:54: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [2]:
def remove_accents(text):
    return unidecode(text)


def get_country_code(country):
    matching_rows = countries_codes.loc[countries_codes['Country'] == country, ' Code']
    return matching_rows.iloc[0] if not matching_rows.empty else None

In [3]:
#PLAYER DATA - MARKET VALUE AND SALARY
leagues_salary = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1//CSVs we use often/latest_capology_data_money_fixed.csv')

leagues_value = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/MarketValuesComplete.csv')


leagues_value_large = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/most_updated_transfermarkt_dataset.csv')

#COUNTRY CODES
countries_codes = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/countries_and_codes.csv')

#FIFA RANKINGS
fifa_rankings = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/fifa_ranking-2023-10-26 - fifa_ranking-2023-07-20.csv')

#MATCH DATA - LINEUPS AND RESULTS
# lineups_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_lineups_data_march_24.csv')
lineups_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_lineups_data_april_29.csv')
results_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_results_data_march_24.csv')

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_19172/3266352280.py:7: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  leagues_value_large = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/most_updated_transfermarkt_dataset.csv')


In [4]:
#USING TERRELLS DATASET
# leagues_value = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/Fixed TransferMarkt Market Values - MarketValues.csv')
leagues_value['Accents Removed'] = leagues_value['Name'].apply(lambda x: unidecode(x))
leagues_value['Team 1 Code'] = leagues_value['Nationality'].apply(get_country_code)
# leagues_value.to_csv("Most Updated Edited Transfermarkt Dataset.csv", index=False)




In [5]:
#HELPER METHODS
##SELENIUM WRAPPER FUNCTION 

def useSeleniumToFindMoney(
        NAME_TO_FIND, #the name it is looking up
        NATIONALITY_INPUT_PLAYER, #natl_test
        DATE_INPUT_MATCH, #input_year_test
        SALARY_BOOLEAN):
    if(SALARY_BOOLEAN == False):
        print('finding salary with selenium')
        marketvalue = AssemblyHelpers.seleniumLookUpValueWrapperFunction(NAME_TO_FIND, NATIONALITY_INPUT_PLAYER, DATE_INPUT_MATCH)
        return marketvalue
    elif(SALARY_BOOLEAN == True):
        salary = AssemblyHelpers.capology_selenium_lookup(NAME_TO_FIND, NATIONALITY_INPUT_PLAYER, DATE_INPUT_MATCH)
        return salary

    else:
        return 0



#MATCH DATE FILTER 
def filter_using_date_of_match(candidate_list_of_names_input, nationality_input, matchdate_Input, SALARY_BOOLEAN): 

    FILTERED_NAMES_USING_MATCH_DATE = AssemblyHelpers.multiNameMatchDateLookup(candidate_list_of_names_input, nationality_input, matchdate_Input) 

    print('in fudom', type(FILTERED_NAMES_USING_MATCH_DATE))
    #1 MATCH
    if isinstance(FILTERED_NAMES_USING_MATCH_DATE, str) or isinstance(FILTERED_NAMES_USING_MATCH_DATE, np.str_): #YELLOW 3
        print('were here', FILTERED_NAMES_USING_MATCH_DATE)
        #print('were in fudom in the right place')
        #NAME_FOR_SELENIUM_SEARCH = FILTERED_NAMES_USING_MATCH_DATE
        # MONEY_FOUND_ONLINE = useSeleniumToFindMoney(NAME_FOR_SELENIUM_SEARCH, nationality_input, matchdate_Input, SALARY_BOOLEAN)
        # print('made it out of using selenium')
        # money_thisplayer = [MONEY_FOUND_ONLINE, f'Online WAS Lookup Required - salary boolean is {SALARY_BOOLEAN}', True, False]
        money_thisplayer = [0, 'Lookup Required', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #MULTIPLE MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) >= 2):
        print(FILTERED_NAMES_USING_MATCH_DATE, type(FILTERED_NAMES_USING_MATCH_DATE), type(FILTERED_NAMES_USING_MATCH_DATE) == str)
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT MULTIPLE RESULTS: {FILTERED_NAMES_USING_MATCH_DATE}', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #0 MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) == 0):
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT 0 RESULTS. BEFORE FILTERING, CANDIDATE NAMES WERE {candidate_list_of_names_input}', True, False]
        return money_thisplayer, ''



#FINDING MONEY INFO USSING PLAYER NAME
def tryToFindPlayerMoney(
        DATABASE_RETURN_MESSAGE, #what the DB said when trying to find money info
        NAME_TO_FIND, #the name it found in the DB (name_match[0] originally)
        NATIONALITY_INPUT_PLAYER, #natl_test
        DATE_INPUT_MATCH, #input_year_test
        SALARY_BOOLEAN, #IF TRUE, SALARIES. OTHERWISE VALUES
        ORIGINAL_SEARCH_NAME #name_match[2]
        ):

    if((DATABASE_RETURN_MESSAGE == "not in DB any of 3 seasons.") or (DATABASE_RETURN_MESSAGE == "asswipe was in DB in before or after season")):
        print('newwwnewnenwenewnwenwenwew', DATABASE_RETURN_MESSAGE)
        
        if DATABASE_RETURN_MESSAGE == 'asswipe was in DB in before or after season':
            money_thisplayer = [0, 'Was in  DB before or after season', True, True]
            return money_thisplayer

        
        
        

        INTERNET_LOOKUP_RESULTING_NAMES = AssemblyHelpers.false_name_match_lookup(ORIGINAL_SEARCH_NAME, NATIONALITY_INPUT_PLAYER)
        NUM_NAMES_REMAINING = len(list(INTERNET_LOOKUP_RESULTING_NAMES))
        print(INTERNET_LOOKUP_RESULTING_NAMES)
        #1 MATCH
        if(NUM_NAMES_REMAINING == 1):
            #FIND INFO USING SELENIUM
            # NAME_FOR_SELENIUM_SEARCH = next(iter(INTERNET_LOOKUP_RESULTING_NAMES))
            # MONEY_FOUND_ONLINE = useSeleniumToFindMoney(NAME_FOR_SELENIUM_SEARCH, NATIONALITY_INPUT_PLAYER, DATE_INPUT_MATCH, SALARY_BOOLEAN)
            # money_thisplayer = [MONEY_FOUND_ONLINE, f'Online WAS Lookup Required - salary boolean is {SALARY_BOOLEAN}', True, False]
            money_thisplayer = [0, 'Lookup Required', True, False]
            return money_thisplayer
        #MULTIPLE MATCHES
        elif(NUM_NAMES_REMAINING >= 2):
            print('goes to FUDOM')
            #FILTER THE NAMES USING THE DATE OF THE MATCH
            list_to_filter_with_match_date = list(INTERNET_LOOKUP_RESULTING_NAMES)
            #NEW FUNCTION 3 
            money_thisplayer, name_results = filter_using_date_of_match(list_to_filter_with_match_date, NATIONALITY_INPUT_PLAYER, DATE_INPUT_MATCH, SALARY_BOOLEAN) 
            return money_thisplayer
        #0 MATCHES
        elif(NUM_NAMES_REMAINING == 0):
            #This is return case 9
            money_thisplayer = [0, 'Online Lookup Required', True, False] 
            return money_thisplayer
    
    else:
        money_thisplayer = [0, 'Online Lookup Required', True, False]
        return money_thisplayer
    


    #old
    # elif(DATABASE_RETURN_MESSAGE == "was in DB in before or after season"):
    #     0==0
    

##OVERARCHING Controller FUNCTION 
def new_overarching(starter_number, name_input, nationality_input, date_input, salary_boolean, competition_input, combination_dict_matches, combination_dict_NON_matches, combination_dict_multi_matches):
    """
    Inputs: Name, Date, Nationality, Salary Boolean 
        salary_boolean
            true – salary
            false – Market Value

    Outputs: Money_thisplayer, name_match
    Outputs to add: error, error type, error msg
    """

    #SETTING INITIAL VALUES
    example_problem = name_input
    natl_test = nationality_input
    input_year_test = date_input

    if 'U-20' in natl_test:
        print('u-20 team')
        natl_test = natl_test.split(' U-20')[0]

    corrrect_code = countries_codes[countries_codes['Country'] == natl_test][' Code'].unique()[0]
    

    if salary_boolean == True:
        0==0
        #dataset_nationality = leagues_salary[leagues_salary['Team 1 Code'] == corrrect_code]['Name'].unique()
    else:
        dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == corrrect_code]['Name'].unique()

    money_thisplayer = [0, 'not done yet', True, False]
    secondmatch_required = False

    #FIRST TRY LOOKING FOR NAME
    name_match = AssemblyHelpers.lookup_name(example_problem, natl_test, input_year_test, salary_boolean, competition_input)
    
    match_case = name_match[1]
    original_db_name_for_search = name_match[2]
    print('match is', name_match)

    

    if(match_case == 'single'):
        NAME_MATCH_FIRST_TRY = name_match[0]
        #print(NAME_MATCH_FIRST_TRY)
        money_thisplayer = AssemblyHelpers.find_money_info_from_name(NAME_MATCH_FIRST_TRY, natl_test, input_year_test, salary_boolean, nullnames_salaries, nullnames_values)

        money_found = money_thisplayer[0]
        search_db_return_case = money_thisplayer[1]
        online_lookup_required = money_thisplayer[2]

        if ((money_found == 0) & (online_lookup_required == True)):         #call the find_money_info_extended once you write it. inputs are = (NAME_MATCH_FIRST_TRY, natl_test, input_year_test, salary_boolean, name_match[2])
            #NEW FUNCTION 1
            if '.' in input_year_test:
                yearstr = input_year_test.split(".")[2]
                full_num = int('20' + yearstr)
            elif '-' in input_year_test:
                if len(input_year_test.split("-")[2]) == 4:
                    yearstr = input_year_test.split("-")[2] #this is a 4 digit number
                    full_num = int(yearstr)
                elif len(input_year_test.split("-")[2]) == 2:
                    if len(input_year_test.split("-")[0]) == 4:
                        yearstr = input_year_test.split("-")[0]
                        full_num = int(yearstr)
                    else:
                        yearstr = input_year_test.split("-")[2] #this is a 2 digit number
                        full_num = int('20' + yearstr)
                else:
                    print('broke on year parsing', input_year_test.split("-"))
                # elif len(input_year_test.split("-")[0]) == 4:
                    # yearstr = input_year_test.split("-")[0]
                    # full_num = yearstr
            
            #print(f'singi trying {NAME_MATCH_FIRST_TRY} and {int('20' + yearstr)}')
            player_bigvalue_dataset = leagues_value_large[(leagues_value_large['Name'] == NAME_MATCH_FIRST_TRY) & (leagues_value_large['Season'] == full_num)].reset_index()
            
            if len(player_bigvalue_dataset) >= 1:
                #print(player_bigvalue_dataset.at[0, 'Market Value'])
                #money_thisplayer[0] =
                #print('working youre good')
                money_thisplayer = [player_bigvalue_dataset.at[0, 'Market Value'], 'Used Large Dataset', False, False]
                if player_bigvalue_dataset.at[0, 'Market Value'] == 0:
                        #money_thisplayer[2] = True
                        money_thisplayer[1] = 'Large Dataset had 0'
            else:
                if len(leagues_value_large[((leagues_value_large['Name'] == NAME_MATCH_FIRST_TRY) & (leagues_value_large['Season'] == full_num - 1)) | 
                                       ((leagues_value_large['Name'] == NAME_MATCH_FIRST_TRY) & (leagues_value_large['Season'] == full_num+ 1))]) >= 2:
                    money_thisplayer = [0, 'Not in either dataset for currect season but present in both seasons around match', False, True]
                    # leagues_value_large[((leagues_value_large['Name'] == NAME_MATCH_FIRST_TRY) & (leagues_value_large['Season'] == full_num - 1)) | 
                    #                    ((leagues_value_large['Name'] == NAME_MATCH_FIRST_TRY) & (leagues_value_large['Season'] == full_num + 1))]['Market Value'].mean()
                else:
                    if ((leagues_value[leagues_value['Name'] == NAME_MATCH_FIRST_TRY]['Season'].max() == (full_num - 1)) & (leagues_value_large[leagues_value_large['Name'] == NAME_MATCH_FIRST_TRY]['Season'].max() == (full_num - 1))):
                        money_last_season = leagues_value[(leagues_value['Name'] == NAME_MATCH_FIRST_TRY) & (leagues_value['Season'] == (full_num - 1))]['Market Value'].max()
                        money_thisplayer = [money_last_season, f'Data Ended before season : {full_num}', False, False]
                    else:
                        money_thisplayer = tryToFindPlayerMoney(search_db_return_case, NAME_MATCH_FIRST_TRY, natl_test, input_year_test, salary_boolean, original_db_name_for_search)
            
        elif(search_db_return_case == 'working'):
            0==0
            #return to the end basically
        elif(search_db_return_case == 'working - imputed'):
            0==0
            #when ur done this should be removed. go into the function find_money_info and remove case
                    
    elif(match_case == 'multiple'):
        print('multi match')

        if '.' in input_year_test:
            yearstr = input_year_test.split(".")[2]
            full_num = int('20' + yearstr)
        elif '-' in input_year_test:
            if len(input_year_test.split("-")[2]) == 4:
                yearstr = input_year_test.split("-")[2] #this is a 4 digit number
                full_num = int(yearstr)
            elif len(input_year_test.split("-")[2]) == 2:
                if len(input_year_test.split("-")[0]) == 4:
                    yearstr = input_year_test.split("-")[0]
                    full_num = int(yearstr)
                else:
                    yearstr = input_year_test.split("-")[2] #this is a 2 digit number
                    full_num = int('20' + yearstr)

        combination_key = (example_problem, full_num)

        if combination_key in combination_dict_matches:

            print('combo is in matches dict')
            #correct_name = combination_dict_matches[combination_key]
            #print('correct name is ', correct_name)

            SINGLE_NAME_MATCH_CORRECT_SEASON = combination_dict_matches[combination_key]
                
            money_thisplayer = AssemblyHelpers.find_money_info_from_name(SINGLE_NAME_MATCH_CORRECT_SEASON[0], natl_test, input_year_test, salary_boolean, nullnames_salaries, nullnames_values) 

            money_found = money_thisplayer[0]
            search_db_return_case = money_thisplayer[1]
            online_lookup_required = money_thisplayer[2]

            if ((money_found == 0) & (online_lookup_required == True)):  
                #NEW FUNCTION 1     
                if '.' in input_year_test:
                    yearstr = input_year_test.split(".")[2]
                    full_num = int('20' + yearstr)
                elif '-' in input_year_test:
                    if len(input_year_test.split("-")[2]) == 4:
                        yearstr = input_year_test.split("-")[2] #this is a 4 digit number
                        full_num = int(yearstr)
                    elif len(input_year_test.split("-")[2]) == 2:
                        if len(input_year_test.split("-")[0]) == 4:
                            yearstr = input_year_test.split("-")[0]
                            full_num = int(yearstr)
                        else:
                            yearstr = input_year_test.split("-")[2] #this is a 2 digit number
                            full_num = int('20' + yearstr)
                    else:
                        print('broke on year parsing', input_year_test.split("-"))
        
                #print(f'multiii trying {SINGLE_NAME_MATCH_CORRECT_SEASON} and {int('20' + yearstr)}')
                if type(SINGLE_NAME_MATCH_CORRECT_SEASON) == list:
                    SINGLE_NAME_MATCH_CORRECT_SEASON = SINGLE_NAME_MATCH_CORRECT_SEASON[0]
                player_bigvalue_dataset = leagues_value_large[(leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value_large['Season'] == full_num)].reset_index()
                
                if len(player_bigvalue_dataset) >= 1:
                    #print(player_bigvalue_dataset.at[0, 'Market Value'])
                    #money_thisplayer[0] =
                    #print('working youre good')
                    money_thisplayer = [player_bigvalue_dataset.at[0, 'Market Value'], 'Used Large Dataset', False, False]
                    if player_bigvalue_dataset.at[0, 'Market Value'] == 0:
                        #money_thisplayer[2] = True
                        money_thisplayer[1] = 'Large Dataset said 0'
                else:
                    if ((leagues_value[leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1)) & (leagues_value_large[leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1))):
                        money_last_season = leagues_value[(leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value['Season'] == (full_num - 1))]['Market Value'].max()
                        money_thisplayer = [money_last_season, f'Data Ended before season : {full_num}', False, False]
                    else:
                        money_thisplayer = tryToFindPlayerMoney(search_db_return_case, SINGLE_NAME_MATCH_CORRECT_SEASON, natl_test, input_year_test, salary_boolean, original_db_name_for_search)
                        
            elif(search_db_return_case == 'working'):
                0==0
                #return to the end basically
            elif(search_db_return_case == 'working - imputed'):
                0==0
                #when ur done this should be removed. go into the function find_money_info and remove case
            
            #in this situation, you need to do find money info from name. 
            #use the name that is in the dict 
        elif combination_key in combination_dict_NON_matches:
            print('combination in non-matches dict')#if you find it in the non matches dictionary
            #save the list of names it returned 
            #then return money thisplayer 
            NAMES_MATCHES_LIST_CORRECT_SEASON = combination_dict_NON_matches[combination_key]
            money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT 0 RESULTS. BEFORE FILTERING, ORIGINAL NAME WAS {example_problem}', True, False]
        elif combination_key in combination_dict_multi_matches:
            #stop the loop here 
            filtered_names_match_date = combination_dict_multi_matches[combination_key]
            money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT MULTIPLE RESULTS: {filtered_names_match_date}', True, False]
        else:
            print('combo is not in matches, multi-matches or non-matches dict')
        
            NAMES_MATCHES_LIST = name_match[0]
            
            
            #FILTER THE NAMES USING THE YEAR OF THE MATCH
            NAMES_MATCHES_LIST_CORRECT_SEASON = AssemblyHelpers.filter_candidates_using_year(starter_number, NAMES_MATCHES_LIST, natl_test, input_year_test, salary_boolean)
            

            NUMBER_OF_NAMES_MATCHING = len(NAMES_MATCHES_LIST_CORRECT_SEASON)
            #print(f'left is {NUMBER_OF_NAMES_MATCHING}')
            
            if(NUMBER_OF_NAMES_MATCHING == 1): #YELLOW 2
                
                
                SINGLE_NAME_MATCH_CORRECT_SEASON = NAMES_MATCHES_LIST_CORRECT_SEASON[0]
                
                money_thisplayer = AssemblyHelpers.find_money_info_from_name(SINGLE_NAME_MATCH_CORRECT_SEASON, natl_test, input_year_test, salary_boolean, nullnames_salaries, nullnames_values) 

                money_found = money_thisplayer[0]
                search_db_return_case = money_thisplayer[1]
                online_lookup_required = money_thisplayer[2]

                if ((money_found == 0) & (online_lookup_required == True)):  
                    #NEW FUNCTION 1     
                    if '.' in input_year_test:
                        yearstr = input_year_test.split(".")[2]
                        full_num = int('20' + yearstr)
                    elif '-' in input_year_test:
                        if len(input_year_test.split("-")[2]) == 4:
                            yearstr = input_year_test.split("-")[2] #this is a 4 digit number
                            full_num = int(yearstr)
                        elif len(input_year_test.split("-")[2]) == 2:
                            if len(input_year_test.split("-")[0]) == 4:
                                yearstr = input_year_test.split("-")[0]
                                full_num = int(yearstr)
                            else:
                                yearstr = input_year_test.split("-")[2] #this is a 2 digit number
                                full_num = int('20' + yearstr)
                        else:
                            print('broke on year parsing', input_year_test.split("-"))
            
                    #print(f'multiii trying {SINGLE_NAME_MATCH_CORRECT_SEASON} and {int('20' + yearstr)}')
                    print(full_num, SINGLE_NAME_MATCH_CORRECT_SEASON)
                    player_bigvalue_dataset = leagues_value_large[(leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value_large['Season'] == full_num)].reset_index()
                    
                    if len(player_bigvalue_dataset) >= 1:
                        #print(player_bigvalue_dataset.at[0, 'Market Value'])
                        #money_thisplayer[0] =
                        #print('working youre good')
                        money_thisplayer = [player_bigvalue_dataset.at[0, 'Market Value'], 'Used Large Dataset', False, False]
                        if player_bigvalue_dataset.at[0, 'Market Value'] == 0:
                            #money_thisplayer[2] = True
                            money_thisplayer[1] = 'Large Dataset said 0'
                    else:
                        if ((leagues_value[leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1)) & (leagues_value_large[leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1))):
                            money_last_season = leagues_value[(leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value['Season'] == (full_num - 1))]['Market Value'].max()
                            money_thisplayer = [money_last_season, f'Data Ended before season : {full_num}', False, False]
                        else:
                            money_thisplayer = tryToFindPlayerMoney(search_db_return_case, SINGLE_NAME_MATCH_CORRECT_SEASON, natl_test, input_year_test, salary_boolean, original_db_name_for_search)
                            
                elif(search_db_return_case == 'working'):
                    0==0
                    #return to the end basically
                elif(search_db_return_case == 'working - imputed'):
                    0==0
                    #when ur done this should be removed. go into the function find_money_info and remove case
                
            elif(NUMBER_OF_NAMES_MATCHING >= 2):
                #use match date to filter 
                #print('list of names is ', NAMES_MATCHES_LIST_CORRECT_SEASON)
                #print('we are here', NUMBER_OF_NAMES_MATCHING)
                if('Olympics' in competition_input):
                    money_thisplayer = [0, F'Olympics row. Couldn\'t filter names using Transfermarkt. remaining matches were {NAMES_MATCHES_LIST_CORRECT_SEASON}', True, False]
                    #the filtering is not going to work 
                else:
                
                    filtered_names_match_date = AssemblyHelpers.multiNameMatchDateLookup(NAMES_MATCHES_LIST_CORRECT_SEASON, natl_test, input_year_test) 

                    #1 MATCH
                    if(type(filtered_names_match_date) == str): #YELLOW 3
                        
                        if '.' in input_year_test:
                            yearstr = input_year_test.split(".")[2]
                            full_num = int('20' + yearstr)
                        elif '-' in input_year_test:
                            if len(input_year_test.split("-")[2]) == 4:
                                yearstr = input_year_test.split("-")[2] #this is a 4 digit number
                                full_num = int(yearstr)
                            elif len(input_year_test.split("-")[2]) == 2:
                                if len(input_year_test.split("-")[0]) == 4:
                                    yearstr = input_year_test.split("-")[0]
                                    full_num = int(yearstr)
                                else:
                                    yearstr = input_year_test.split("-")[2] #this is a 2 digit number
                                    full_num = int('20' + yearstr)
                        #print('we here', filtered_names_match_date, full_num, example_problem)
                        combination_key = (example_problem, full_num)
                        combination_dict_matches[combination_key] = [filtered_names_match_date]
                        print(f'added key {combination_key} to dict. value is {filtered_names_match_date}')
                        #at this point, example problem and full_num become a combination 
                        #that combination should be a key, and the value should be filtered_names_match_date
                        #add a new key+value combination to the dictionary here. 

                        NAME_MATCH_SINGLE = filtered_names_match_date

                        #name_match[0] = NAME_MATCH_SINGLE
                        #print('we hereeeee', NAME_MATCH_SINGLE)
                        money_thisplayer = AssemblyHelpers.find_money_info_from_name(NAME_MATCH_SINGLE, natl_test, input_year_test, salary_boolean, nullnames_salaries, nullnames_values) 
                        
                        money_found = money_thisplayer[0]
                        search_db_return_case = money_thisplayer[1]
                        online_lookup_required = money_thisplayer[2]

                        if ((money_found == 0) & (online_lookup_required == True)):         #call the find_money_info_extended once you write it. inputs are = (NAME_MATCH_FIRST_TRY, natl_test, input_year_test, salary_boolean, name_match[2])
                            # NAME_FOR_SELENIUM_SEARCH = NAME_MATCH_SINGLE
                            # MONEY_FOUND_ONLINE = useSeleniumToFindMoney(NAME_FOR_SELENIUM_SEARCH, natl_test, input_year_test, salary_boolean)
                            # money_thisplayer = [MONEY_FOUND_ONLINE, f'Online WAS Lookup Required - salary boolean is {salary_boolean}', True, False]

                            if len(leagues_value_large[(leagues_value_large['Name'] == NAME_MATCH_SINGLE) & (leagues_value_large['Season'] == full_num)]) >= 1:
                                money_thatseason_large_db = leagues_value_large[(leagues_value_large['Name'] == NAME_MATCH_SINGLE) & (leagues_value_large['Season'] == full_num)]['Market Value'].max()
                                money_thisplayer = [money_thatseason_large_db, 'Used Large Dataset', False, False]

                            elif ((leagues_value[leagues_value['Name'] == NAME_MATCH_SINGLE]['Season'].max() == (full_num - 1)) & (leagues_value_large[leagues_value_large['Name'] == NAME_MATCH_SINGLE]['Season'].max() == (full_num - 1))):
                                money_last_season = leagues_value[(leagues_value['Name'] == NAME_MATCH_SINGLE) & (leagues_value['Season'] == (full_num - 1))]['Market Value'].max()
                                money_thisplayer = [money_last_season, f'Data Ended before season : {full_num}', False, False]
                            else:
                                #money_thisplayer = tryToFindPlayerMoney(search_db_return_case, NAME_MATCH_FIRST_TRY, natl_test, input_year_test, salary_boolean, original_db_name_for_search)
                                money_thisplayer = [0, 'Lookup Required', True, False]
                            #return money_thisplayer
                            
                        elif(search_db_return_case == 'working'):
                            0==0
                            #return to the end basically
                        elif(search_db_return_case == 'working - imputed'):
                            0==0
                            #when ur done this should be removed. go into the function find_money_info and remove case


                    #MULTIPLE MATCHES
                    elif(len(filtered_names_match_date) >= 2): #RED 6

                        print('added a row to multi matches dictionary.')
                        combination_key_multi_match = (example_problem, full_num)
                        combination_dict_multi_matches[combination_key_multi_match] = [filtered_names_match_date]
                        
                        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT MULTIPLE RESULTS: {filtered_names_match_date}', True, False]
                        #NNNNEW - add to the dictionary here 
                        #return money_thisplayer, name_match
                    #0 MATCHES
                    elif(len(filtered_names_match_date) == 0): #RED 5
                        #print('we are here. bahrain case')
                        if '.' in input_year_test:
                            yearstr = input_year_test.split(".")[2]
                            full_num = int('20' + yearstr)
                        elif '-' in input_year_test:
                            if len(input_year_test.split("-")[2]) == 4:
                                yearstr = input_year_test.split("-")[2] #this is a 4 digit number
                                full_num = int(yearstr)
                            elif len(input_year_test.split("-")[2]) == 2:
                                if len(input_year_test.split("-")[0]) == 4:
                                    yearstr = input_year_test.split("-")[0]
                                    full_num = int(yearstr)
                                else:
                                    yearstr = input_year_test.split("-")[2] #this is a 2 digit number
                                    full_num = int('20' + yearstr)

                        combination_key_nomatch = (example_problem, full_num)
                        combination_dict_NON_matches[combination_key_nomatch] = [NAMES_MATCHES_LIST_CORRECT_SEASON]
                        print(f'added a row to non matches dict. player was {example_problem} in {full_num}.')
                        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT 0 RESULTS. BEFORE FILTERING, ORIGINAL NAME WAS {example_problem}', True, False]
                        #return money_thisplayer, name_match
                    
            elif(NUMBER_OF_NAMES_MATCHING == 0):#RETURN CASE 4

                print('here rc4')
                
                money_thisplayer, name_results = filter_using_date_of_match(NAMES_MATCHES_LIST, natl_test, input_year_test, salary_boolean) 
                if money_thisplayer == [0, 'Lookup Required', True, False]:
                    SINGLE_NAME_MATCH_CORRECT_SEASON = name_results
                    combination_key = (example_problem, full_num)
                    combination_dict_matches[combination_key] = [SINGLE_NAME_MATCH_CORRECT_SEASON]
                    print(f'added key {combination_key} to dict. value is {SINGLE_NAME_MATCH_CORRECT_SEASON}')
                    player_bigvalue_dataset = leagues_value_large[(leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value_large['Season'] == full_num)].reset_index()
                    
                    if len(player_bigvalue_dataset) >= 1:
                        #print(player_bigvalue_dataset.at[0, 'Market Value'])
                        #money_thisplayer[0] =
                        #print('working youre good')
                        money_thisplayer = [player_bigvalue_dataset.at[0, 'Market Value'], 'Used Large Dataset', False, False]
                        if player_bigvalue_dataset.at[0, 'Market Value'] == 0:
                            #money_thisplayer[2] = True
                            money_thisplayer[1] = 'Large Dataset said 0'
                    else:
                        if ((leagues_value[leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1)) & (leagues_value_large[leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1))):
                            money_last_season = leagues_value[(leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value['Season'] == (full_num - 1))]['Market Value'].max()
                            money_thisplayer = [money_last_season, f'Data Ended before season : {full_num}', False, False]
                        else:
                            money_thisplayer = tryToFindPlayerMoney(money_thisplayer[1], SINGLE_NAME_MATCH_CORRECT_SEASON, natl_test, input_year_test, salary_boolean, original_db_name_for_search)
                
    

    elif(match_case == 'none'):
        #secondmatch_required = True
        #print('ya 1')

        NAME_TO_SEARCH = name_match[2]

        NAME_FOURTH_TRY = AssemblyHelpers.fourth_try_name_search(NAME_TO_SEARCH, natl_test, salary_boolean)

        MATCH_CASE_FOURTH_TRY = NAME_FOURTH_TRY[0]

        if(MATCH_CASE_FOURTH_TRY == 'single'): #YELLOW 1
            NAME_FOUND = NAME_FOURTH_TRY[1]
            money_thisplayer = AssemblyHelpers.find_money_info_from_name(NAME_FOUND, natl_test, input_year_test, salary_boolean, nullnames_salaries, nullnames_values) 

            money_found = money_thisplayer[0]
            search_db_return_case = money_thisplayer[1]
            online_lookup_required = money_thisplayer[2]

            if '.' in input_year_test:
                yearstr = input_year_test.split(".")[2]
                full_num = int('20' + yearstr)
            elif '-' in input_year_test:
                if len(input_year_test.split("-")[2]) == 4:
                    yearstr = input_year_test.split("-")[2] #this is a 4 digit number
                    full_num = int(yearstr)
                elif len(input_year_test.split("-")[2]) == 2:
                    if len(input_year_test.split("-")[0]) == 4:
                        yearstr = input_year_test.split("-")[0]
                        full_num = int(yearstr)
                    else:
                        yearstr = input_year_test.split("-")[2] #this is a 2 digit number
                        full_num = int('20' + yearstr)

            if ((money_found == 0) & (online_lookup_required == True)): #call the find_money_info_extended once you write it. inputs are = (NAME_MATCH_FIRST_TRY, natl_test, input_year_test, salary_boolean, name_match[2])
                if ((leagues_value[leagues_value['Name'] == NAME_FOUND]['Season'].max() == (full_num - 1)) & (leagues_value_large[leagues_value_large['Name'] == NAME_FOUND]['Season'].max() == (full_num - 1))):
                    money_last_season = leagues_value[(leagues_value['Name'] == NAME_FOUND) & (leagues_value['Season'] == (full_num - 1))]['Market Value'].max()
                    money_thisplayer = [money_last_season, f'Data Ended before season : {full_num}', False, False]
                else:
                    money_thisplayer = tryToFindPlayerMoney(search_db_return_case, NAME_FOUND, natl_test, input_year_test, salary_boolean, original_db_name_for_search)
                
            elif(search_db_return_case == 'working'):
                0==0
                #return to the end basically
            elif(search_db_return_case == 'working - imputed'):
                0==0
                #when ur done this should be removed. go into the function find_money_info and remove case

        elif(MATCH_CASE_FOURTH_TRY == 'multiple'):#RED 3

            print('here 4th try')
    
            LIST_TO_FILTER = NAME_FOURTH_TRY[1]
            money_thisplayer, name_results = filter_using_date_of_match(LIST_TO_FILTER, natl_test, input_year_test, salary_boolean) 
            if money_thisplayer == [0, 'Lookup Required', True, False]:
                SINGLE_NAME_MATCH_CORRECT_SEASON = name_results
                combination_key = (example_problem, full_num)
                combination_dict_matches[combination_key] = [SINGLE_NAME_MATCH_CORRECT_SEASON]
                print(f'added key {combination_key} to dict. value is {SINGLE_NAME_MATCH_CORRECT_SEASON}')
                player_bigvalue_dataset = leagues_value_large[(leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value_large['Season'] == full_num)].reset_index()
                
                if len(player_bigvalue_dataset) >= 1:
                    #print(player_bigvalue_dataset.at[0, 'Market Value'])
                    #money_thisplayer[0] =
                    #print('working youre good')
                    money_thisplayer = [player_bigvalue_dataset.at[0, 'Market Value'], 'Used Large Dataset', False, False]
                    if player_bigvalue_dataset.at[0, 'Market Value'] == 0:
                        #money_thisplayer[2] = True
                        money_thisplayer[1] = 'Large Dataset said 0'
                else:
                    if ((leagues_value[leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1)) & (leagues_value_large[leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1))):
                        money_last_season = leagues_value[(leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value['Season'] == (full_num - 1))]['Market Value'].max()
                        money_thisplayer = [money_last_season, f'Data Ended before season : {full_num}', False, False]
                    else:
                        money_thisplayer = tryToFindPlayerMoney(money_thisplayer[1], SINGLE_NAME_MATCH_CORRECT_SEASON, natl_test, input_year_test, salary_boolean, original_db_name_for_search)
        
        elif(MATCH_CASE_FOURTH_TRY == 'none'): #SELENIUM
            #print('ya 2')
            original_db_name_for_search = name_match[2]

            accents_removed_name = remove_accents(original_db_name_for_search)
            #print(accents_removed_name)
            equal_names = [name for name in dataset_nationality if name == accents_removed_name]
            #(accents_removed_name in dataset_nationality)

            if equal_names:
                0==0
            else:
            #print
                matching_names = [name for name in dataset_nationality if name.endswith(accents_removed_name)]
                #print('weere here', matching_names)
                if len(matching_names) >= 2:
                    0==0 #more than 1 person remains
                elif len(matching_names) == 1:
                    0==0 #single matching name
                    #print('heylo')
                    money_thisplayer = AssemblyHelpers.find_money_info_from_name(matching_names[0], natl_test, input_year_test, salary_boolean, nullnames_salaries, nullnames_values) 
                else:
                    0==0 #no matches 
                    #print('nerr')
                    # MONEY_FOUND_ONLINE_0 = useSeleniumToFindMoney(original_db_name_for_search, natl_test, input_year_test, salary_boolean)
                    # money_thisplayer = [MONEY_FOUND_ONLINE_0, f'Online WAS Lookup Required - salary boolean is {salary_boolean}', True, False]
                    money_thisplayer = [0, 'Lookup Required', True, False]
    else:
        0==0 #RED 7

    return money_thisplayer, name_match


def remove_prefix(string):
    # Define pattern to match digits or '-' at the start of the string
    pattern = r'^[\d-]+'
    
    # Use re.sub to remove the matched pattern from the start of the string
    return re.sub(pattern, '', string)

In [6]:
countries_codes['Accents Removed'] = countries_codes['Country'].apply(remove_accents)


In [7]:
combination_dict_matches = {}
combination_dict_multi_matches = {}

combination_dict_NON_matches = {}
pd.set_option('display.max_columns', None)


In [29]:
# Define the list of values to filter
competition_values = [
    'World Cup 2014, Qualifiers, CONMEBOL',
    'World Cup 2018, Qualifiers, CONMEBOL',
    'World Cup 2022, Qualifiers, CONMEBOL',
    'World Cup 2014, Qualifiers, CONCACAF',
    'World Cup 2018, Qualifiers, CONCACAF',
    'World Cup 2022, Qualifiers, CONCACAF'
]

# Filter the DataFrame
lineups_data_concacaf_conmebol = lineups_data[lineups_data['Competition'].isin(competition_values)].reset_index()

#len(lineups_data_concacaf_conmebol)

In [10]:
combined_df = pd.read_csv('April 25 - CONCACAF & CONMEBOL OUT - DATA CSVs/Combined DF/large_combined_df_new_april_26.csv')

red_group = pd.read_csv('April 25 - CONCACAF & CONMEBOL OUT - DATA CSVs/Red Groups (merged)/CONCACAF & CONMEBOL OUT - Wrong DFs Merged.csv')

pink_cyrillic = pd.read_csv('April 25 - CONCACAF & CONMEBOL OUT - DATA CSVs/Pink Group/CONCACAF & CONMEBOL OUT - Cyrillic_Guys_Nobody_Found.csv')

pink_weird = pd.read_csv('April 25 - CONCACAF & CONMEBOL OUT - DATA CSVs/Pink Group/CONCACAF & CONMEBOL OUT - odd_name_found_0_cases.csv')

pink_lookup = pd.read_csv('April 25 - CONCACAF & CONMEBOL OUT - DATA CSVs/Pink Group/CONCACAF & CONMEBOL OUT - Online lookup required - 0 names found.csv')


/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_9720/1150788479.py:1: DtypeWarning: Columns (12,20) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df = pd.read_csv('April 25 - CONCACAF & CONMEBOL OUT - DATA CSVs/Combined DF/large_combined_df_new_april_26.csv')


In [30]:
dates_og = lineups_data_concacaf_conmebol['Date']


for i in range(0, len(lineups_data_concacaf_conmebol)):
    if lineups_data_concacaf_conmebol.at[i, 'Date'].startswith('Sat '):
        lineups_data_concacaf_conmebol.at[i, 'Date'] = lineups_data_concacaf_conmebol.at[i, 'Date'].split('Sat ')[1]
    elif lineups_data_concacaf_conmebol.at[i, 'Date'].startswith('Sun '):
        lineups_data_concacaf_conmebol.at[i, 'Date'] = lineups_data_concacaf_conmebol.at[i, 'Date'].split('Sun ')[1]
    elif lineups_data_concacaf_conmebol.at[i, 'Date'].startswith('Wed '):
        lineups_data_concacaf_conmebol.at[i, 'Date'] = lineups_data_concacaf_conmebol.at[i, 'Date'].split('Wed ')[1]
    elif lineups_data_concacaf_conmebol.at[i, 'Date'].startswith('Tue '):
        lineups_data_concacaf_conmebol.at[i, 'Date'] = lineups_data_concacaf_conmebol.at[i, 'Date'].split('Tue ')[1]
    elif lineups_data_concacaf_conmebol.at[i, 'Date'].startswith('Thu '):
        lineups_data_concacaf_conmebol.at[i, 'Date'] = lineups_data_concacaf_conmebol.at[i, 'Date'].split('Thu ')[1]
    elif lineups_data_concacaf_conmebol.at[i, 'Date'].startswith('Mon '):
        lineups_data_concacaf_conmebol.at[i, 'Date'] = lineups_data_concacaf_conmebol.at[i, 'Date'].split('Mon ')[1]
    elif lineups_data_concacaf_conmebol.at[i, 'Date'].startswith('Fri '):
        lineups_data_concacaf_conmebol.at[i, 'Date'] = lineups_data_concacaf_conmebol.at[i, 'Date'].split('Fri ')[1]
lineups_data_concacaf_conmebol['Date'] = pd.to_datetime(lineups_data_concacaf_conmebol['Date'])

In [8]:
def add_correct_money(money_column):
    if money_column == 0:
        return 0

    if 'm' in money_column:
        value_rough = money_column.split('m')[0].split('€')[1]
        millinum = (float(value_rough) * 1000000)
        return millinum
    elif 'k' in money_column:
        #print(i, 'k')
        value_rough = money_column.split('k')[0].split('€')[1]
        millinum = (float(value_rough) * 1000)

        return millinum
    elif '-' in money_column:
        return 0
    else:
        return 0

def in_season_around(player_id, curr_season, age):
    if age == "Not Listed": age = 30
    age = int(age)
    year_before = int(curr_season) -1
    year_after = int(curr_season) + 1
    before_season = leagues_value[(leagues_value['PlayerID'] == player_id) & (leagues_value['Season'] == year_before)] #leagues_value_large #leagues_value
    after_season =  leagues_value[(leagues_value['PlayerID'] == player_id) & (leagues_value['Season'] == year_after)]
    if before_season.empty and after_season.empty:
        return -1, "Both Empty"
    if not before_season.empty and after_season.empty:
        if before_season.iloc[0]['Market Value'] != "-":
            if before_season.iloc[0]['Market Value'] == 0:
                return 0, "Only Before + Zero Before"
            else:
                if age >= 30:
                    return (float(add_correct_money(before_season.iloc[0]['Market Value'])) * .8), "Only Before" #add_correct_money
                else:
                    return (float(add_correct_money(before_season.iloc[0]['Market Value'])) * 1.2), "Only Before" #add_correct_money
        else:
            return 0, "Only Before X"
    if not after_season.empty and before_season.empty:
        
        if after_season.iloc[0]['Market Value'] != "-":
            if after_season.iloc[0]['Market Value'] == 0:
                return 0, "Only After + Zero After"
            else:
                if age <= 30:
                    return (float(add_correct_money(after_season.iloc[0]['Market Value'])) * .8), "Only After"
                else:
                    return (float(add_correct_money(after_season.iloc[0]['Market Value'])) * 1.2), "Only After"
        else:
            return 0, "Only After X"
    if not before_season.empty and not after_season.empty:
        #print('hi')
        before_season_val = before_season.iloc[0]['Market Value']
        after_season_val = after_season.iloc[0]['Market Value']
        #print(before_season_val, after_season_val)
        if before_season_val != "-" and after_season_val != "-":
            # if type(before_season_val) == str:
            #     before_season_val = add_correct_money(before_season_val)
            # if type(after_season_val) == str:
            #     after_season_val = add_correct_money(after_season_val)
            return ((float(add_correct_money(before_season.iloc[0]['Market Value'])) + float(add_correct_money(after_season.iloc[0]['Market Value'])) / 2)), "Used Average of Season Before and After"
        elif before_season_val != "-" and after_season_val == "-":
            if age >= 30:
                return (float(add_correct_money(before_season.iloc[0]['Market Value']) * .8)), "Used Season Before * .8 But Had Both"
            else:
                return (float(add_correct_money(before_season.iloc[0]['Market Value']) * 1.2)), "Used Season Before * 1.2 But Had Both"
        elif before_season_val == "-" and after_season_val != "-":
            if age <= 30:
                return (float(add_correct_money(after_season.iloc[0]['Market Value']) * .8)), "Used Season After * .8 But Had Both"
            else:
                return (float(add_correct_money(after_season.iloc[0]['Market Value']) * 1.2)), "Used Season After * 1.2 But Had Both"
        else:
            return 0, "Had Both but Both Were Empty"


def find_in_transfermarkt(player_name, season, country_code, database_name):
    #print(player_name, country_code, season)
    row = database_name[(database_name['Name'] == player_name) & (database_name['Season'] == season) & (database_name['Team 1 Code'] == country_code)]
    one_up_row = database_name[(database_name['Name'] == player_name) & (database_name['Season'] == int(season)+1) & (database_name['Team 1 Code'] == country_code)]
    one_down_row = database_name[(database_name['Name'] == player_name) & (database_name['Season'] == int(season)-1) & (database_name['Team 1 Code'] == country_code)]
    print(len(row), len(one_down_row), len(one_up_row))
    final_value = 0
    if not row.empty:
        #print('yeeea')
        player_id = row.iloc[0]['PlayerID']
        age = row.iloc[0]['Age']
        marketval = row.iloc[0]['Market Value']
        if type(marketval) == str:
            #print('der')
            if marketval == '-':
                marketval = 0
            elif '€' in marketval:
                marketval = add_correct_money(marketval)
            marketval = float((marketval)) #add_correct_money
        if marketval != 0:
            #print('wooo')
            #print('***', player_name, marketval)
            return marketval, "Found in Curr Season"
        else:
            #print(marketval)
            new_mv, reason = in_season_around(player_id, season, age)
            print(player_name, new_mv, reason)
            if new_mv == -1 or new_mv == 0 or new_mv == "-":
                return 0, reason
            else:
                return new_mv, reason
    elif not one_up_row.empty:
        player_id = one_up_row.iloc[0]['PlayerID']
        age = one_up_row.iloc[0]['Age']
        new_mv, reason = in_season_around(player_id, season, age)
        if new_mv == -1 or new_mv == 0 or new_mv == "-":
            return 0, reason
        else:
            return new_mv, reason
    elif not one_down_row.empty:
        player_id = one_down_row.iloc[0]['PlayerID']
        age = one_down_row.iloc[0]['Age']
        new_mv, reason = in_season_around(player_id, season, age)
        if new_mv == -1 or new_mv == 0 or new_mv == "-":
            return 0, reason
        else:
            return new_mv, reason
    else:
        return 0, "Did Not Find in Curr Season, One Up, or One Down"
    

In [153]:
lineups_data_concacaf_conmebol['Status New'] = ''

In [166]:
lookup_dataset_1 = pd.DataFrame()


not_found_dataset = pd.DataFrame()


In [167]:
counter = 0
salary_boolean = False
for k in range(0, len(lineups_data_concacaf_conmebol)): #lineups_data_concacaf_conmebol

    nationality_this = lineups_data_concacaf_conmebol.at[k, 'TeamName']
    competition_this = lineups_data_concacaf_conmebol.at[k, 'Competition']
    #nationality_this = lineups_data.at[k, 'Match']
    date_this = lineups_data_concacaf_conmebol.at[k, 'Date']

    
    country_code_this = lineups_data_concacaf_conmebol.at[k, 'Code']
    date_this = str(date_this)
    if ' 00:00:00' in date_this:
        date_this = date_this.split(' 00:00:00')[0]
    print(k, nationality_this, date_this)
    for i in range(2, 13): 
        number = -999
        reason = ''

        column = lineups_data_concacaf_conmebol.columns[i]
        player_this = remove_prefix(lineups_data_concacaf_conmebol.at[k, f'{column}'])
        if player_this[0] == ' ':
            player_this = player_this[1:]

        if player_this == '-':
            0==0
        else:
            #print(player_this)
            
            if '-' in date_this:
                if len(date_this.split("-")[2]) == 4:
                    yearstr = date_this.split("-")[2] #this is a 4 digit number
                    full_num = int(yearstr)
                elif len(date_this.split("-")[2]) == 2:
                    if len(date_this.split("-")[0]) == 4:
                        yearstr = date_this.split("-")[0]
                        full_num = int(yearstr)
                    else:
                        yearstr = date_this.split("-")[2] #this is a 2 digit number
                        full_num = int('20' + yearstr)
        dataset_nationality = leagues_value[(leagues_value['Team 1 Code'] == country_code_this) & 
                                        (leagues_value['Season'].isin([full_num, full_num + 1, full_num - 1]))]['Name'].unique()
        if player_this in dataset_nationality:
            0==0
            
            number, reason = find_in_transfermarkt(player_this, full_num, country_code_this, leagues_value)
            #print(player_this, reason, add_correct_money(number))

            #add a new row to lookup dataset 1 
            if type(number) == float:
                0==0
            else:
                number = add_correct_money(number)
            
            lineups_data_concacaf_conmebol.at[k, 'Status New'] = 'Done'

            data_row = {
                'Match ID': lineups_data_concacaf_conmebol.at[k, 'Match ID'],
                'Competition': competition_this,
                'Match': lineups_data_concacaf_conmebol.at[k, 'Match'],
                'Date': date_this,
                'Nationality': nationality_this,
                'Team Country Code': country_code_this,
                'Season': full_num,
                'Status': 'SUCCESS',
                'Name': player_this, #this will be zero if a fail happens 
                'Market Value': number,
                'Lookup Still Required?': False,
                'Lookup Return Case': reason,
                'Impute Required?': False,
                'Name(s) Found': player_this,
                'ORIGINAL JERSEY': lineups_data_concacaf_conmebol.at[k, f'{column}'],
                'Match Case': 'single' #single, multiple, none or cyrillic
            }
            lookup_dataset_1 = pd.concat([lookup_dataset_1, pd.DataFrame([data_row])], ignore_index=True)
            
        else:
            print(player_this, 'nope')
            # counter += 1
            dataset_nationality_all_years = leagues_value[(leagues_value['Team 1 Code'] == country_code_this)]['Name'].unique()
            if player_this in dataset_nationality_all_years:
                
                print('***Found Him', player_this)
                data_row = {
                    'Match ID': lineups_data_concacaf_conmebol.at[k, 'Match ID'],
                    'Competition': competition_this,
                    'Match': lineups_data_concacaf_conmebol.at[k, 'Match'],
                    'Date': date_this,
                    'Nationality': nationality_this,
                    'Team Country Code': country_code_this,
                    'Season': full_num,
                    'Status': 'FAIL',
                    'Name': player_this, #this will be zero if a fail happens 
                    'Market Value': 0,
                    'Lookup Still Required?': True,
                    'Lookup Return Case': 'Did Not Find in Curr Season, One Up, or One Down', 
                    'Impute Required?': False,
                    'Name(s) Found': player_this,
                    'ORIGINAL JERSEY': lineups_data_concacaf_conmebol.at[k, f'{column}'],
                    'Match Case': 'single' #single, multiple, none or cyrillic
                }        
                not_found_dataset = pd.concat([not_found_dataset, pd.DataFrame([data_row])], ignore_index=True)    
            else:
                print('DID NOT FIND ', player_this)
                counter += 1
                data_row = {
                    'Match ID': lineups_data_concacaf_conmebol.at[k, 'Match ID'],
                    'Competition': competition_this,
                    'Match': lineups_data_concacaf_conmebol.at[k, 'Match'],
                    'Date': date_this,
                    'Nationality': nationality_this,
                    'Team Country Code': country_code_this,
                    'Season': full_num,
                    'Status': 'FAIL',
                    'Name': player_this, #this will be zero if a fail happens 
                    'Market Value': 0,
                    'Lookup Still Required?': True,
                    'Lookup Return Case': 'Did Not Find in any Season', #Did Not Find in Curr Season, One Up, or One Down
                    'Impute Required?': False,
                    'Name(s) Found': player_this,
                    'ORIGINAL JERSEY': lineups_data_concacaf_conmebol.at[k, f'{column}'],
                    'Match Case': 'single' #single, multiple, none or cyrillic
                }        
                not_found_dataset = pd.concat([not_found_dataset, pd.DataFrame([data_row])], ignore_index=True)     
            
    #dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == country_code_this]['Name'].unique()
        #make the dataset nationality equal to only the season before , current or after 

print(counter)



0 Honduras 2013-02-06
1 USA 2013-02-06
2 Panama 2013-02-07
3 Costa Rica 2013-02-07
4 Mexico 2013-02-07
5 Jamaica 2013-02-07
Donovan Ricketts nope
***Found Him Donovan Ricketts
Marvin Elliott nope
***Found Him Marvin Elliott
Jermaine Johnson nope
***Found Him Jermaine Johnson
6 Honduras 2013-03-22
7 Mexico 2013-03-22
8 Jamaica 2013-03-23
Donovan Ricketts nope
***Found Him Donovan Ricketts
Marvin Elliott nope
***Found Him Marvin Elliott
Jermaine Beckford nope
***Found Him Jermaine Beckford
9 Panama 2013-03-23
10 USA 2013-03-23
11 Costa Rica 2013-03-23
12 Costa Rica 2013-03-27
13 Jamaica 2013-03-27
Donovan Ricketts nope
***Found Him Donovan Ricketts
Marvin Elliott nope
***Found Him Marvin Elliott
Jermaine Beckford nope
***Found Him Jermaine Beckford
14 Panama 2013-03-27
15 Honduras 2013-03-27
16 Mexico 2013-03-27
17 USA 2013-03-27
18 Jamaica 2013-06-08
Donovan Ricketts nope
***Found Him Donovan Ricketts
Marvin Elliott nope
***Found Him Marvin Elliott
Jermaine Johnson nope
***Found Him Jer

In [ ]:
#TRACKING MY WORK 

#the guys it didnt find WERE in the dataset. just not in the years we wanted. 

In [183]:
for i in range(0, len(not_found_dataset)):
    if not_found_dataset.at[i, 'Nationality'] == 'British Virgin Islands':
        not_found_dataset.at[i, 'Team Country Code'] = ' BVI'

In [9]:
not_found_dataset

NameError: name 'not_found_dataset' is not defined

In [204]:
for index,row in not_found_dataset.iterrows():
    if row['Nationality'] == 'British Virgin Islands': 
        name = row['Name(s) Found']
        country_code_this = row['Team Country Code']
        full_num = row['Season']
        dataset_nationality = leagues_value[(leagues_value['Team 1 Code'] == country_code_this) & 
                                        (leagues_value['Season'].isin([full_num, full_num + 1, full_num - 1]))]['Name'].unique()
        if name in dataset_nationality:
            print(index, row['Nationality'])
            
            number, reason = find_in_transfermarkt(player_this, full_num, country_code_this, leagues_value)
            #print(player_this, reason, add_correct_money(number))

            #add a new row to lookup dataset 1 
            if type(number) == float:
                0==0
            else:
                number = add_correct_money(number)
            correct_lineups_row = lineups_data_concacaf_conmebol[(lineups_data_concacaf_conmebol['Match ID'] == row['Match ID']) & (lineups_data_concacaf_conmebol['TeamName'] == row['Nationality'])]
            if len(correct_lineups_row) == 1:
                #print(correct_lineups_row.index)
                lineups_data_concacaf_conmebol.at[correct_lineups_row.index[0], 'Status New'] = 'Done'

            data_row = {
                'Match ID': row['Match ID'],
                'Competition': row['Competition'],
                'Match': row['Match'],
                'Date': row['Date'],
                'Nationality': row['Nationality'],
                'Team Country Code': country_code_this,
                'Season': full_num,
                'Status': 'SUCCESS',
                'Name': name, #this will be zero if a fail happens 
                'Market Value': number,
                'Lookup Still Required?': False,
                'Lookup Return Case': reason,
                'Impute Required?': False,
                'Name(s) Found': name,
                'ORIGINAL JERSEY': lineups_data_concacaf_conmebol.at[k, f'{column}'],
                'Match Case': 'single' #single, multiple, none or cyrillic
            }
            lookup_dataset_1 = pd.concat([lookup_dataset_1, pd.DataFrame([data_row])], ignore_index=True)



22 British Virgin Islands
23 British Virgin Islands
24 British Virgin Islands
25 British Virgin Islands
26 British Virgin Islands
27 British Virgin Islands
28 British Virgin Islands
29 British Virgin Islands
30 British Virgin Islands
31 British Virgin Islands
32 British Virgin Islands
33 British Virgin Islands
34 British Virgin Islands
35 British Virgin Islands
36 British Virgin Islands
37 British Virgin Islands
38 British Virgin Islands
39 British Virgin Islands
40 British Virgin Islands
41 British Virgin Islands
42 British Virgin Islands
43 British Virgin Islands
48 British Virgin Islands
49 British Virgin Islands
50 British Virgin Islands
51 British Virgin Islands
52 British Virgin Islands
53 British Virgin Islands
54 British Virgin Islands
55 British Virgin Islands
56 British Virgin Islands
57 British Virgin Islands
58 British Virgin Islands
59 British Virgin Islands
60 British Virgin Islands
61 British Virgin Islands
62 British Virgin Islands
63 British Virgin Islands
64 British V

In [243]:
not_found_dataset = not_found_dataset.reset_index()

In [246]:
for i in range(0, len(not_found_dataset)):
    if not_found_dataset.at[i, 'Status'] == 'FAIL':
        not_found_dataset.at[i, 'Status'] = 'FAIL - NEW'

not_found_dataset = not_found_dataset.drop(columns={'index'})

In [247]:
not_found_dataset

,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case
0,3934,"World Cup 2014, Qualifiers, CONCACAF",Mexico vs Jamaica,2013-02-07,Jamaica,JM,2013,FAIL - NEW,Donovan Ricketts,0,True,"Did Not Find in Curr Season, One Up, or One Down",False,Donovan Ricketts,1 Donovan Ricketts,single
1,3934,"World Cup 2014, Qualifiers, CONCACAF",Mexico vs Jamaica,2013-02-07,Jamaica,JM,2013,FAIL - NEW,Marvin Elliott,0,True,"Did Not Find in Curr Season, One Up, or One Down",False,Marvin Elliott,4 Marvin Elliott,single
2,3934,"World Cup 2014, Qualifiers, CONCACAF",Mexico vs Jamaica,2013-02-07,Jamaica,JM,2013,FAIL - NEW,Jermaine Johnson,0,True,"Did Not Find in Curr Season, One Up, or One Down",False,Jermaine Johnson,8 Jermaine Johnson,single
3,3938,"World Cup 2014, Qualifiers, CONCACAF",Jamaica vs Panama,2013-03-23,Jamaica,JM,2013,FAIL - NEW,Donovan Ricketts,0,True,"Did Not Find in Curr Season, One Up, or One Down",False,Donovan Ricketts,1 Donovan Ricketts,single
4,3938,"World Cup 2014, Qualifiers, CONCACAF",Jamaica vs Panama,2013-03-23,Jamaica,JM,2013,FAIL - NEW,Marvin Elliott,0,True,"Did Not Find in Curr Season, One Up, or One Down",False,Marvin Elliott,4 Marvin Elliott,single
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,4445,"World Cup 2022, Qualifiers, CONMEBOL",Brazil vs Chile,2022-03-24,Chile,CL,2022,FAIL - NEW,Mauricio Isla,0,True,"Did Not Find in Curr Season, One Up, or One Down",False,Mauricio Isla,4 Mauricio Isla,single
87,4445,"World Cup 2022, Qualifiers, CONMEBOL",Brazil vs Chile,2022-03-24,Chile,CL,2022,FAIL - NEW,Eduardo Vargas,0,True,"Did Not Find in Curr Season, One Up, or One Down",False,Eduardo Vargas,11 Eduardo Vargas,single
88,4446,"World Cup 2022, Qualifiers, CONMEBOL",Paraguay vs Ecuador,2022-03-24,Ecuador,EC,2022,FAIL - NEW,Joao Rojas,0,True,"Did Not Find in Curr Season, One Up, or One Down",False,Joao Rojas,10 Joao Rojas,single
89,4452,"World Cup 2022, Qualifiers, CONMEBOL",Chile vs Uruguay,2022-03-29,Chile,CL,2022,FAIL - NEW,Mauricio Isla,0,True,"Did Not Find in Curr Season, One Up, or One Down",False,Mauricio Isla,4 Mauricio Isla,single


In [248]:
lookup_dataset_final = pd.concat([lookup_dataset_1, not_found_dataset], ignore_index=True)

In [251]:
combined_df_plus = pd.concat([combined_df, lookup_dataset_final], ignore_index=True)

In [228]:
combined_df_plus['Market Value New'] = combined_df_plus['Market Value New'].fillna(0)

combined_df_plus[combined_df_plus['Market Value New'] != 0]

,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case,To Remove,Status New,Market Value New
57628,2181,"World Cup Qualifiers, Africa, 2022",Tanzania vs Burundi,08-09-2019,Tanzania,TZ,2019,SUCCESS,Ramadhani Awam Kabwili,0,False,Found in Curr Season,False,Hassan Kessy,H Kessy,single,False,NaN,75000.0
57731,3546,"African Cup of Nations, 2019",Tanzania vs Algeria,01-07-2019,Tanzania,TZ,2019,SUCCESS,Ramadhani Awam Kabwili,0,False,Found in Curr Season,False,Hassan Kessy,H Kessy,single,False,Done,75000.0
57732,3551,"African Cup of Nations, 2019",Kenya vs Tanzania,27-06-2019,Tanzania,TZ,2019,SUCCESS,Ramadhani Awam Kabwili,0,False,Found in Curr Season,False,Hassan Kessy,H Kessy,single,False,Done,75000.0
57733,3574,"African Cup of Nations, 2019",Senegal vs Tanzania,23-06-2019,Tanzania,TZ,2019,SUCCESS,Ramadhani Awam Kabwili,0,False,Found in Curr Season,False,Hassan Kessy,H Kessy,single,False,Done,75000.0
57752,1915,"World Cup Qualifiers, Africa, 2014",Burkina Faso vs Gabon,07-09-2013,Burkina Faso,BF,2013,SUCCESS,Mbah Koné,0,False,Found in Curr Season,False,Mohamed Koffi,M Koffi,single,False,Done,400000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70011,2211,"World Cup Qualifiers, Africa, 2026",Liberia vs Malawi,17-11-2023,Liberia,LR,2023,SUCCESS,Tonia Tisdell,0,False,Only Before,True,Terrence Tisdell,T Tisdell,single,False,Done,240000.0
71752,3497,"African Cup of Nations, 2015",Cote d'Ivoire vs Ghana,08-02-2015,Cote d'Ivoire,CI,2015,SUCCESS,Vakoun Bayo,0,False,Only Before,False,Boubacar Copa,Boubacar Copa,single,False,Done,1200000.0
72118,11,"U-20 World Cup, 2015",Ukraine U-20 vs Senegal U-20,10.06.15,Senegal,SN,2015,SUCCESS,['Alhassane Sylla'],0,False,Found in Curr Season,False,['Alhassane Sylla'],Alhassane Sylla,Single,False,Done,3800000.0
72785,1273,Copa America 2019,Argentina vs Chile,06.07.19,Chile,CL,2019,SUCCESS,['Víctor Dávila'],2100000,False,Found in Curr Season,False,Paulo Díaz,P Diaz,Single,False,Done,4000000.0


In [268]:
combined_df_plus['Status'].value_counts()

Status
SUCCESS       87922
FAIL            203
FAIL - NEW       91
Name: count, dtype: int64

In [267]:
combined_df_plus['Status New'] = combined_df_plus['Status New'].fillna('')
combined_df_plus['Status New'].value_counts()

Status New
        87833
Done      383
Name: count, dtype: int64

In [269]:
for index, row in combined_df_plus.iterrows():
    if row['Status New'] == 'Done':
        if row['Market Value New'] != 0:
            
            combined_df_plus.at[index, 'Market Value'] = row['Market Value New']
            #print(index, row['Market Value New'], combined_df_plus.at[index, 'Market Value'])
        else:
            print(index, row['Market Value New'], combined_df_plus.at[index, 'Market Value'])

57553 0.0 0
57554 0.0 0
57555 0.0 0
57556 0.0 0
57557 0.0 0
57558 0.0 0
57559 0.0 0
57560 0.0 0
57561 0.0 0
57562 0.0 0
57563 0.0 0
57564 0.0 0
57565 0.0 0
57566 0.0 0
57567 0.0 0
57568 0.0 0
57569 0.0 0
57570 0.0 0
57571 0.0 0
57572 0.0 0
57573 0.0 0
57574 0.0 0
57575 0.0 0
57576 0.0 0
57577 0.0 0
57578 0.0 0
57579 0.0 0
57580 0.0 0
57581 0.0 0
57582 0.0 0
57583 0.0 0
57584 0.0 0
57585 0.0 0
57586 0.0 0
57587 0.0 0
57588 0.0 0
57589 0.0 0
57590 0.0 0
57591 0.0 0
57592 0.0 0
57593 0.0 0
57594 0.0 0
57595 0.0 0
57596 0.0 0
57597 0.0 0
57598 0.0 0
57599 0.0 0
57600 0.0 0
57601 0.0 0
57602 0.0 0
57603 0.0 0
57604 0.0 0
57605 0.0 0
57606 0.0 0
57607 0.0 0
57608 0.0 0
57609 0.0 0
57610 0.0 0
57611 0.0 0
57612 0.0 0
57613 0.0 0
57614 0.0 0
57615 0.0 0
57616 0.0 0
57617 0.0 0
57618 0.0 0
57619 0.0 0
57620 0.0 0
57621 0.0 0
57622 0.0 0
57623 0.0 0
57624 0.0 0
57625 0.0 0
57626 0.0 0
57627 0.0 0
57629 0.0 0
57630 0.0 0
57631 0.0 0
57632 0.0 0
57633 0.0 0
57634 0.0 0
57635 0.0 0
57636 0.0 0
5763

In [277]:
combined_df_plus = combined_df_plus.drop(columns={'To Remove', 'Status New', 'Market Value New'})
combined_df_plus

,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case
0,3133,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Anthony Moris,400000.0,False,working,False,Anthony Moris,A Moris,single
1,3133,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Laurent Jans,1200000.0,False,working,False,Laurent Jans,L Jans,single
2,3133,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Kevin Malget,150000.0,False,working,False,Kevin Malget,K Malget,single
3,3133,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Mario Mutsch,125000.0,False,working,False,Mario Mutsch,M Mutsch,single
4,3133,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Chris Philipps,1250000.0,False,working,False,Chris Philipps,C Philipps,single
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88211,4445,"World Cup 2022, Qualifiers, CONMEBOL",Brazil vs Chile,2022-03-24,Chile,CL,2022,FAIL - NEW,Mauricio Isla,0.0,True,"Did Not Find in Curr Season, One Up, or One Down",False,Mauricio Isla,4 Mauricio Isla,single
88212,4445,"World Cup 2022, Qualifiers, CONMEBOL",Brazil vs Chile,2022-03-24,Chile,CL,2022,FAIL - NEW,Eduardo Vargas,0.0,True,"Did Not Find in Curr Season, One Up, or One Down",False,Eduardo Vargas,11 Eduardo Vargas,single
88213,4446,"World Cup 2022, Qualifiers, CONMEBOL",Paraguay vs Ecuador,2022-03-24,Ecuador,EC,2022,FAIL - NEW,Joao Rojas,0.0,True,"Did Not Find in Curr Season, One Up, or One Down",False,Joao Rojas,10 Joao Rojas,single
88214,4452,"World Cup 2022, Qualifiers, CONMEBOL",Chile vs Uruguay,2022-03-29,Chile,CL,2022,FAIL - NEW,Mauricio Isla,0.0,True,"Did Not Find in Curr Season, One Up, or One Down",False,Mauricio Isla,4 Mauricio Isla,single


In [278]:
#combined_df_plus.to_csv('combined_df_new_april30.csv')
combined_df_plus.to_csv('combined_df_may_1.csv')